In [1]:
import os

In [2]:
%pwd

'h:\\Projects\\mlops\\first\\pizza-steak-end-to-end\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'h:\\Projects\\mlops\\first\\pizza-steak-end-to-end'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [6]:
from PizzaSteakClassifier.constants import *
from PizzaSteakClassifier.utils.common import read_yaml_file, create_directories


In [7]:
X = read_yaml_file(CONFIG_FILE_PATH)

[2024-01-05 22:30:10,269 - INFO - common - YAML file loaded successfully: config\config.yaml]


In [8]:
X.base_model_preparation.root_dir

'artifacts/base_model_preparation'

In [9]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        
        self.config = read_yaml_file(config_file_path)
        self.params = read_yaml_file(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.base_model_preparation
        params = self.params

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir = Path(config.root_dir),
            base_model_path = Path(config.base_model_path),
            updated_base_model_path = Path(config.updated_base_model_path),
            params_image_size = params.IMAGE_SIZE,
            params_learning_rate = params.LEARNING_RATE,
            params_include_top = params.INCLUDE_TOP,
            params_weights = params.WEIGHTS,
            params_classes = params.CLASSES
        )

        return prepare_base_model_config

In [10]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

[2024-01-05 22:30:16,943 - WARNING - module_wrapper - From h:\Projects\mlops\first\pizza-steak-end-to-end\.venv\Lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]


In [15]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
    
    def download_base_model(self):
        self.model = tf.keras.applications.EfficientNetB0(include_top = self.config.params_include_top,
                                                          weights = self.config.params_weights,
                                                          classes = self.config.params_classes)
        self.save_base_model(path = self.config.base_model_path,
                             model = self.model)
    
    def prepare_full_model(self):
        base_model = self.model
        base_model.trainable = False
        input_shape = tuple(self.config.params_image_size)
        num_classes = self.config.params_classes
        learning_rate = self.config.params_learning_rate


        input_layer = tf.keras.layers.Input(shape = input_shape)
        x = base_model(input_layer, training = False)
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = tf.keras.layers.Dense(units=20, activation = 'relu')(x)
        output_layer = tf.keras.layers.Dense(units = num_classes, activation = 'softmax')(x)
        

        full_model = tf.keras.Model(input_layer, output_layer)

        full_model.compile(
            optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate),
            loss = tf.keras.losses.CategoricalCrossentropy(),
            metrics = ['accuracy']
        )

        full_model.summary()

        return full_model
    
    @staticmethod
    def save_base_model(path: Path, model: tf.keras.models.Model):
        model.save(path)
    
    def update_base_model(self):
        self.full_model = self.prepare_full_model()
        self.save_base_model(path = self.config.updated_base_model_path,
                             model = self.full_model)


In [16]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.download_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-01-05 22:31:52,457 - INFO - common - YAML file loaded successfully: config\config.yaml]
[2024-01-05 22:31:52,460 - INFO - common - YAML file loaded successfully: params.yaml]
[2024-01-05 22:31:52,462 - INFO - common - Directory created: artifacts]
[2024-01-05 22:31:52,463 - INFO - common - Directory created: artifacts/base_model_preparation]
[2024-01-05 22:31:54,432 - WARNING - saving_utils - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional  (None, None, None, 1280   4049571   
 )                           )                                   
                                 